В файле movies.csv лежит база фильмов. Название фильма записано во втором столбце title.
Разбейте названия фильмов на отдельные слова и посчитайте какое слово встречается чаще всего.

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('word_count').getOrCreate()

21/10/11 13:37:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [21]:

with open ('movies.csv') as file:
    words = []
    file.readline() # пропускаем первую строку с названиями столбцов
    
    titles = [line.strip().split(',')[1].split(' ')[:-1] for line in file] # [:-1] - убираем из названий год выпуска, считая, что он всегда указан
    [words.extend(w) for w in titles]

df_words = spark.sparkContext.parallelize(words)

df_words_ones = df_words.map(lambda x: (x,1))
df_words_count = df_words_ones.reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending=False)

df_words_count.take(10)



[('of', 946),
 ('the', 911),
 ('The', 411),
 ('and', 328),
 ('in', 245),
 ('to', 190),
 ('a', 174),
 ('&', 105),
 ('on', 96),
 ('(a.k.a.', 91)]